In [ ]:
import pyodbc
import pandas as pd
from IPython.core.display import display


ModuleNotFoundError: No module named 'pyodbc'

In [93]:
EODReport = """Select * from reclusereports.dbo.eodreport Where tradetype!='Forward' and ObservationDate = '2022-03-02'"""
OverridesRep="""Select * from reclusereports.dbo.Overrides Where ObservationDate = '2022-03-02' and product='Total'"""
OverridesRepPD="""Select * from reclusereports.dbo.Overrides Where ObservationDate = '2022-03-01' and product='Total'"""
connectionstring = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=sbawsdtsql01q.secbenefit.com\\DODS_QA;DATABASE=DODSValuation;integrated security=Yes;Trusted_Connection=Yes"
connection = pyodbc.connect(connectionstring)
EODdf = pd.read_sql(EODReport, connection)
overridedf=pd.read_sql(OverridesRep, connection)
OverridesRepPDdf=pd.read_sql(OverridesRepPD, connection)
connection.close()


In [94]:
Presentdf=pd.DataFrame()

Getoverridecolumn=overridedf[["GroupName","Contract","Override"]].replace({'Groupname':{'FEDL01 IRSOIS':'IRS','US0003M IRSFS':'IRS'}})

Presentdf=Getoverridecolumn.copy()
Presentdf=Presentdf.replace({'GroupName':{'FEDL01 IRSOIS':'IRS','US0003M IRSFS':'IRS'}})
Presentdf=Presentdf.groupby(['GroupName'],as_index=False).sum()
print(Presentdf)


                     GroupName  Contract  Override
0           BNPIHDPL Call 1 YR        86        10
1           BNPIHDPL Call 2 YR        75         0
2           BXIIMWM7 Call 1 YR       173       115
3           BXIIMWM7 Call 2 YR       276       138
4                DRLBLEND Call       418         1
5                       Future        51        23
6                          IRS        63        17
7                MSUSMSDA Call       434         1
8   MSUSMSDA Synthetic Forward         9         0
9            SPLV5UT Call 1 YR       261        55
10           SPLV5UT Call 2 YR       331       118
11          SPMARC5P Call 1 YR       267       124
12          SPMARC5P Call 2 YR       479       115
13                    SPX Call       150         2
14             SPX Call Spread       304        32
15           SPX Monthly Asian        52        25
16         SPX Monthly Cliquet       444       336
17                         TRS        76         7
18            UBSMPI Call 1 YR 

In [95]:
#Rule 1 (Pct Rule)
Rule1=EODdf.copy()
Rule1=Rule1.replace({'PAMName':{'BNPIHDPL-EUROPEAN - 1 Yr': 'BNPIHDPL Call 1 YR','BNPIHDPL-EUROPEAN - 2 Yr': 'BNPIHDPL Call 2 YR','Null':'IRS','BXIIMWM7-EUROPEAN - 1 Yr':'BXIIMWM7 Call 1 YR','BXIIMWM7-EUROPEAN - 2 Yr':'BXIIMWM7 Call 2 YR','MSUSMSDA-EUROPEAN':'MSUSMSDA Call','MSUSMSDA-SYNTHETICFORWARD':'MSUSMSDA Synthetic Forward','S&P 500 - ASIAN CAPPED':'SPX Monthly Asian','S&P 500 - CLIQUET':'SPX Monthly Cliquet','S&P 500 - Vanilla Call':'SPX Call','S&P 500 - CAPPED':'SPX Call Spread','TRANSPARENT VALUE BLENDED':'DRLBLEND Call','SPLV5UT-EUROPEAN 1 Yr':'SPLV5UT Call 1 YR','SPLV5UT-EUROPEAN 2 Yr':'SPLV5UT Call 2 YR','SPMARC5P Index - 1 Yr':'SPMARC5P Call 1 YR','SPMARC5P Index - 2 Yr':'SPMARC5P Call 2 YR','UBSMPI-EUROPEAN':'UBSMPI Call 1 YR','UBSMPI-EUROPEAN 2 Yr':'UBSMPI Call 2 YR'}})
Rule1=Rule1.rename(columns={'PAMName':'GroupName'})
Rule1['Rule1 tolerance']=((Rule1['InternalModel']-Rule1['CounterpartyMark'])/Rule1['CounterpartyMark']).abs()
Rule1=Rule1[Rule1['Rule1 tolerance']>=pd.to_numeric(Rule1['rule1'])]
Rule1=Rule1.groupby(['GroupName'],as_index=False).count()
Rule1fin=Rule1[['GroupName','ObservationDate']]
Presentdf=pd.merge(Presentdf,Rule1fin,on='GroupName',how='left').fillna(0)
Presentdf['Rule 1']=Presentdf['ObservationDate']/Presentdf['Override']
Presentdf['Rule 1'] = Presentdf['Rule 1'].fillna(0)
Presentdf=Presentdf.drop(columns=['ObservationDate'])

print(Presentdf)

                     GroupName  Contract  Override    Rule 1
0           BNPIHDPL Call 1 YR        86        10  1.000000
1           BNPIHDPL Call 2 YR        75         0  0.000000
2           BXIIMWM7 Call 1 YR       173       115  1.000000
3           BXIIMWM7 Call 2 YR       276       138  0.528986
4                DRLBLEND Call       418         1  0.000000
5                       Future        51        23  0.000000
6                          IRS        63        17  0.000000
7                MSUSMSDA Call       434         1  1.000000
8   MSUSMSDA Synthetic Forward         9         0  0.000000
9            SPLV5UT Call 1 YR       261        55  1.000000
10           SPLV5UT Call 2 YR       331       118  0.110169
11          SPMARC5P Call 1 YR       267       124  1.000000
12          SPMARC5P Call 2 YR       479       115  0.973913
13                    SPX Call       150         2  0.000000
14             SPX Call Spread       304        32  0.062500
15           SPX Monthly

In [96]:
#Rule 2 (Basis Rule)
Rule2=EODdf.copy()
Rule2=Rule2.replace({'PAMName':{'BNPIHDPL-EUROPEAN - 1 Yr': 'BNPIHDPL Call 1 YR','BNPIHDPL-EUROPEAN - 2 Yr': 'BNPIHDPL Call 2 YR','Null':'IRS','BXIIMWM7-EUROPEAN - 1 Yr':'BXIIMWM7 Call 1 YR','BXIIMWM7-EUROPEAN - 2 Yr':'BXIIMWM7 Call 2 YR','MSUSMSDA-EUROPEAN':'MSUSMSDA Call','MSUSMSDA-SYNTHETICFORWARD':'MSUSMSDA Synthetic Forward','S&P 500 - ASIAN CAPPED':'SPX Monthly Asian','S&P 500 - CLIQUET':'SPX Monthly Cliquet','S&P 500 - Vanilla Call':'SPX Call','S&P 500 - CAPPED':'SPX Call Spread','TRANSPARENT VALUE BLENDED':'DRLBLEND Call','SPLV5UT-EUROPEAN 1 Yr':'SPLV5UT Call 1 YR','SPLV5UT-EUROPEAN 2 Yr':'SPLV5UT Call 2 YR','SPMARC5P Index - 1 Yr':'SPMARC5P Call 1 YR','SPMARC5P Index - 2 Yr':'SPMARC5P Call 2 YR','UBSMPI-EUROPEAN':'UBSMPI Call 1 YR','UBSMPI-EUROPEAN 2 Yr':'UBSMPI Call 2 YR'}})
Rule2=Rule2.rename(columns={'PAMName':'GroupName'})
Rule2['Rule2 tolerance']=((Rule2['InternalModel']-Rule2['CounterpartyMark'])/Rule2['Notional']*10000).abs()
Rule2=Rule2[Rule2['Rule2 tolerance']>=pd.to_numeric(Rule2['rule2'])]
Rule2=Rule2.groupby(['GroupName'],as_index=False).count()
Rule2fin=Rule2[['GroupName','ObservationDate']]
Presentdf=pd.merge(Presentdf,Rule2fin,on='GroupName',how='left').fillna(0)
Presentdf['Rule 2']=Presentdf['ObservationDate']/Presentdf['Override']
Presentdf=Presentdf.drop(columns=['ObservationDate'])
#Presentdf=Presentdf.style.format({'Rule 2': "{:,.2%}"})
print(Presentdf)

                     GroupName  Contract  Override    Rule 1    Rule 2
0           BNPIHDPL Call 1 YR        86        10  1.000000  0.000000
1           BNPIHDPL Call 2 YR        75         0  0.000000       NaN
2           BXIIMWM7 Call 1 YR       173       115  1.000000  0.330435
3           BXIIMWM7 Call 2 YR       276       138  0.528986  1.000000
4                DRLBLEND Call       418         1  0.000000  1.000000
5                       Future        51        23  0.000000  0.000000
6                          IRS        63        17  0.000000  0.000000
7                MSUSMSDA Call       434         1  1.000000  1.000000
8   MSUSMSDA Synthetic Forward         9         0  0.000000       NaN
9            SPLV5UT Call 1 YR       261        55  1.000000  0.454545
10           SPLV5UT Call 2 YR       331       118  0.110169  1.000000
11          SPMARC5P Call 1 YR       267       124  1.000000  0.040323
12          SPMARC5P Call 2 YR       479       115  0.973913  0.156522
13    

In [97]:
#Get Override MKVL
OVRIDEMK=EODdf.copy()
OVRIDEMK=OVRIDEMK.replace({'PAMName':{'BNPIHDPL-EUROPEAN - 1 Yr': 'BNPIHDPL Call 1 YR','BNPIHDPL-EUROPEAN - 2 Yr': 'BNPIHDPL Call 2 YR','Null':'IRS','BXIIMWM7-EUROPEAN - 1 Yr':'BXIIMWM7 Call 1 YR','BXIIMWM7-EUROPEAN - 2 Yr':'BXIIMWM7 Call 2 YR','MSUSMSDA-EUROPEAN':'MSUSMSDA Call','MSUSMSDA-SYNTHETICFORWARD':'MSUSMSDA Synthetic Forward','S&P 500 - ASIAN CAPPED':'SPX Monthly Asian','S&P 500 - CLIQUET':'SPX Monthly Cliquet','S&P 500 - Vanilla Call':'SPX Call','S&P 500 - CAPPED':'SPX Call Spread','TRANSPARENT VALUE BLENDED':'DRLBLEND Call','SPLV5UT-EUROPEAN 1 Yr':'SPLV5UT Call 1 YR','SPLV5UT-EUROPEAN 2 Yr':'SPLV5UT Call 2 YR','SPMARC5P Index - 1 Yr':'SPMARC5P Call 1 YR','SPMARC5P Index - 2 Yr':'SPMARC5P Call 2 YR','UBSMPI-EUROPEAN':'UBSMPI Call 1 YR','UBSMPI-EUROPEAN 2 Yr':'UBSMPI Call 2 YR'}})
OVRIDEMK=OVRIDEMK.rename(columns={'PAMName':'GroupName'})
OVRIDEMK=OVRIDEMK.groupby(['GroupName','Source'],as_index=False).sum()
OVRIDEMK=OVRIDEMK[OVRIDEMK['Source']=='Internal']
OVRIDEMK=OVRIDEMK.rename(columns={'CounterpartyMark':'Counterparty Override MV','InternalModel':'SBL Override MV'})
OVRIDEMKcols=OVRIDEMK[["GroupName","Counterparty Override MV","SBL Override MV"]]
Presentdf=Presentdf.merge(OVRIDEMKcols,left_on='GroupName',right_on='GroupName')
print(Presentdf)

              GroupName  Contract  Override    Rule 1    Rule 2  \
0    BNPIHDPL Call 1 YR        86        10  1.000000  0.000000   
1    BXIIMWM7 Call 1 YR       173       115  1.000000  0.330435   
2    BXIIMWM7 Call 2 YR       276       138  0.528986  1.000000   
3         DRLBLEND Call       418         1  0.000000  1.000000   
4         MSUSMSDA Call       434         1  1.000000  1.000000   
5     SPLV5UT Call 1 YR       261        55  1.000000  0.454545   
6     SPLV5UT Call 2 YR       331       118  0.110169  1.000000   
7    SPMARC5P Call 1 YR       267       124  1.000000  0.040323   
8    SPMARC5P Call 2 YR       479       115  0.973913  0.156522   
9              SPX Call       150         2  0.000000  1.500000   
10      SPX Call Spread       304        32  0.062500  1.000000   
11    SPX Monthly Asian        52        25  0.360000  1.000000   
12  SPX Monthly Cliquet       444       336  0.776786  0.270833   
13                  TRS        76         7  0.000000  0.14285

In [98]:
#Get Total MVs
EODride=EODdf.copy()
EODride=EODride.replace({'PAMName':{'BNPIHDPL-EUROPEAN - 1 Yr': 'BNPIHDPL Call 1 YR','BNPIHDPL-EUROPEAN - 2 Yr': 'BNPIHDPL Call 2 YR','Null':'IRS','BXIIMWM7-EUROPEAN - 1 Yr':'BXIIMWM7 Call 1 YR','BXIIMWM7-EUROPEAN - 2 Yr':'BXIIMWM7 Call 2 YR','MSUSMSDA-EUROPEAN':'MSUSMSDA Call','MSUSMSDA-SYNTHETICFORWARD':'MSUSMSDA Synthetic Forward','S&P 500 - ASIAN CAPPED':'SPX Monthly Asian','S&P 500 - CLIQUET':'SPX Monthly Cliquet','S&P 500 - Vanilla Call':'SPX Call','S&P 500 - CAPPED':'SPX Call Spread','TRANSPARENT VALUE BLENDED':'DRLBLEND Call','SPLV5UT-EUROPEAN 1 Yr':'SPLV5UT Call 1 YR','SPLV5UT-EUROPEAN 2 Yr':'SPLV5UT Call 2 YR','SPMARC5P Index - 1 Yr':'SPMARC5P Call 1 YR','SPMARC5P Index - 2 Yr':'SPMARC5P Call 2 YR','UBSMPI-EUROPEAN':'UBSMPI Call 1 YR','UBSMPI-EUROPEAN 2 Yr':'UBSMPI Call 2 YR'}})
EODride=EODride.rename(columns={'PAMName':'GroupName'})
EODride=EODride.groupby(by='GroupName',as_index=False).sum()
EODrideselect=EODride[["GroupName","CounterpartyMark","InternalModel"]]
Presentdf=Presentdf.merge(EODrideselect,left_on='GroupName',right_on='GroupName')
print(Presentdf)

              GroupName  Contract  Override    Rule 1    Rule 2  \
0    BNPIHDPL Call 1 YR        86        10  1.000000  0.000000   
1    BXIIMWM7 Call 1 YR       173       115  1.000000  0.330435   
2    BXIIMWM7 Call 2 YR       276       138  0.528986  1.000000   
3         DRLBLEND Call       418         1  0.000000  1.000000   
4         MSUSMSDA Call       434         1  1.000000  1.000000   
5     SPLV5UT Call 1 YR       261        55  1.000000  0.454545   
6     SPLV5UT Call 2 YR       331       118  0.110169  1.000000   
7    SPMARC5P Call 1 YR       267       124  1.000000  0.040323   
8    SPMARC5P Call 2 YR       479       115  0.973913  0.156522   
9              SPX Call       150         2  0.000000  1.500000   
10      SPX Call Spread       304        32  0.062500  1.000000   
11    SPX Monthly Asian        52        25  0.360000  1.000000   
12  SPX Monthly Cliquet       444       336  0.776786  0.270833   
13                  TRS        76         7  0.000000  0.14285

In [99]:
#Total Override PD
PDOverride=OverridesRepPDdf[["GroupName","Override_MV"]]
PDOverride=PDOverride.rename(columns={'Override_MV':'PD Override MV'})
Presentdf=pd.merge(Presentdf,PDOverride,on='GroupName',how='left').fillna(0)
print(Presentdf)

              GroupName  Contract  Override    Rule 1    Rule 2  \
0    BNPIHDPL Call 1 YR        86        10  1.000000  0.000000   
1    BXIIMWM7 Call 1 YR       173       115  1.000000  0.330435   
2    BXIIMWM7 Call 2 YR       276       138  0.528986  1.000000   
3         DRLBLEND Call       418         1  0.000000  1.000000   
4         MSUSMSDA Call       434         1  1.000000  1.000000   
5     SPLV5UT Call 1 YR       261        55  1.000000  0.454545   
6     SPLV5UT Call 2 YR       331       118  0.110169  1.000000   
7    SPMARC5P Call 1 YR       267       124  1.000000  0.040323   
8    SPMARC5P Call 2 YR       479       115  0.973913  0.156522   
9              SPX Call       150         2  0.000000  1.500000   
10      SPX Call Spread       304        32  0.062500  1.000000   
11    SPX Monthly Asian        52        25  0.360000  1.000000   
12  SPX Monthly Cliquet       444       336  0.776786  0.270833   
13                  TRS        76         7  0.000000  0.14285

In [100]:
#Total Override CD
CDOverride=overridedf[["GroupName","Override_MV"]]
CDOverride=CDOverride.rename(columns={'Override_MV':'CD Override MV'})
Presentdf=pd.merge(Presentdf,CDOverride,on='GroupName',how='left').fillna(0)
print(Presentdf)

              GroupName  Contract  Override    Rule 1    Rule 2  \
0    BNPIHDPL Call 1 YR        86        10  1.000000  0.000000   
1    BXIIMWM7 Call 1 YR       173       115  1.000000  0.330435   
2    BXIIMWM7 Call 2 YR       276       138  0.528986  1.000000   
3         DRLBLEND Call       418         1  0.000000  1.000000   
4         MSUSMSDA Call       434         1  1.000000  1.000000   
5     SPLV5UT Call 1 YR       261        55  1.000000  0.454545   
6     SPLV5UT Call 2 YR       331       118  0.110169  1.000000   
7    SPMARC5P Call 1 YR       267       124  1.000000  0.040323   
8    SPMARC5P Call 2 YR       479       115  0.973913  0.156522   
9              SPX Call       150         2  0.000000  1.500000   
10      SPX Call Spread       304        32  0.062500  1.000000   
11    SPX Monthly Asian        52        25  0.360000  1.000000   
12  SPX Monthly Cliquet       444       336  0.776786  0.270833   
13                  TRS        76         7  0.000000  0.14285

In [101]:
#CD-PD overrode
Presentdf['T vs T-1 Difference']=(Presentdf['CD Override MV']-Presentdf['PD Override MV'])
print(Presentdf)

              GroupName  Contract  Override    Rule 1    Rule 2  \
0    BNPIHDPL Call 1 YR        86        10  1.000000  0.000000   
1    BXIIMWM7 Call 1 YR       173       115  1.000000  0.330435   
2    BXIIMWM7 Call 2 YR       276       138  0.528986  1.000000   
3         DRLBLEND Call       418         1  0.000000  1.000000   
4         MSUSMSDA Call       434         1  1.000000  1.000000   
5     SPLV5UT Call 1 YR       261        55  1.000000  0.454545   
6     SPLV5UT Call 2 YR       331       118  0.110169  1.000000   
7    SPMARC5P Call 1 YR       267       124  1.000000  0.040323   
8    SPMARC5P Call 2 YR       479       115  0.973913  0.156522   
9              SPX Call       150         2  0.000000  1.500000   
10      SPX Call Spread       304        32  0.062500  1.000000   
11    SPX Monthly Asian        52        25  0.360000  1.000000   
12  SPX Monthly Cliquet       444       336  0.776786  0.270833   
13                  TRS        76         7  0.000000  0.14285

In [ ]:
#% of Val Override

In [102]:
#To excel
Presentdf.to_csv('Overridestestdata.csv')
#Presentdf=Presentdf.style.format({'Rule 1': "{:,.2%}"})